In [3]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
from time import sleep
import random

In [4]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

print(mt5.version())

account=51666596
password="s!K48dU5!qgrOI"
server="ICMarketsEU-Demo"

authorized=mt5.login(account, server=server,password=password)  # the terminal database password is applied if connection data is set to be remembered
if authorized:
    print("connected to account #{}".format(account))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# now connect to another trading account specifying the password
authorized=mt5.login(account, server=server,password=password)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.45
(500, 4153, '22 Jan 2024')
connected to account #51666596
AccountInfo(login=51666596, trade_mode=0, leverage=30, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=25000.0, credit=0.0, profit=0.0, equity=25000.0, margin=0.0, margin_free=25000.0, margin_level=0.0, margin_so_call=100.0, margin_so_so=50.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Azarm Hamid', server='ICMarketsEU-Demo', currency='USD', company='IC Markets (EU) Ltd')
Show account_info()._asdict():
  login=51666596
  trade_mode=0
  leverage=30
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=25000.0
  credit=0.0
  profit=0.0
  equity=25000.0
  margin=0.0
  margin_free=25000.0
  margin_level=0.0
  margin_so_

# define stop limit position maker funcs

In [5]:
def buy(symbol,price,volume,sl):
  #  point=mt5.symbol_info(symbol).point
  #  price = mt5.symbol_info_tick(symbol).ask
  #  sl = price - point*10*stop
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": sl,
    #  "tp": mt5.symbol_info_tick(symbol).ask+100*point,
        "deviation": 10,
        "magic": 234000,
    #    "comment": "python script",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)

    return result

In [6]:
def buy_stop(symbol,volume,stop,distance):
    point = mt5.symbol_info(symbol).point
    ask = mt5.symbol_info_tick(symbol).ask
    price = ask + distance*point
    sl = price - stop*point
    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": price,
        "deviation": 2,
        "sl": sl,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    result = mt5.order_send(request)

    return result

In [28]:
buy_stop('US30',5.1,10,150)

OrderSendResult(retcode=10009, deal=0, order=597506847, volume=5.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3972263686, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='US30', volume=5.1, price=38937.31, stoplimit=0.0, sl=38937.21, tp=0.0, deviation=2, type=4, type_filling=2, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [10]:
def trail_sl_buy(symbol,ticket):
    bid = mt5.symbol_info_tick(symbol).bid
    request = {
            "action": mt5.TRADE_ACTION_SLTP,
            "symbol": symbol,
            "type": mt5.ORDER_TYPE_BUY,
            "position": ticket,
            "sl": bid ,
            "deviation": 2,
            "magic": 234000,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

    result = mt5.order_send(request)

    return result.comment

In [11]:
def sell(symbol,volume,stop):
    point=mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    sl = price + point*10*stop
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
    #  "tp": mt5.symbol_info_tick(symbol).ask+100*point,
        "deviation": 10,
        "magic": 234000,
    #    "comment": "python script",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)

    return result

In [12]:
def sell_stop(symbol,volume,stop,distance):
    point = mt5.symbol_info(symbol).point
    bid = mt5.symbol_info_tick(symbol).bid
    price = bid - distance*point
    sl = price + stop*point
    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": price,
        "deviation": 2,
        "sl": sl,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    result = mt5.order_send(request)

    return result

In [26]:
buy_stop('US30',0.99,10,200)

OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='US30', volume=0.99, price=38913.31, stoplimit=0.0, sl=38913.21, tp=0.0, deviation=2, type=4, type_filling=2, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [13]:
def trail_sl_sell(symbol,ticket):
    ask = mt5.symbol_info_tick(symbol).ask
    request = {
            "action": mt5.TRADE_ACTION_SLTP,
            "symbol": symbol,
            "type": mt5.ORDER_TYPE_BUY,
            "position": ticket,
            "sl": ask ,
            "deviation": 2,
            "magic": 234000,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

    result = mt5.order_send(request)

    return result.comment

In [14]:
def cancel_order(ticket):
    request = {
        "action": mt5.TRADE_ACTION_REMOVE,
        "order": ticket,
    }
    order_result = mt5.order_send(request)


    return order_result

In [186]:
cancel_order(153380686)[7]

'Request executed'

In [ ]:
while True:
    Buy = buy_stop('US30',volume=5.1,stop=10,distance=150)
    Sell = sell_stop('US30',volume=5.1,stop=10,distance=150)

    while not mt5.positions_get(symbol='US30'):
        continue

    while mt5.positions_total() == 1:
        if (mt5.positions_total() == 1) and (mt5.positions_get(symbol='US30')[0][0] == Buy[2]): # open buy position
            cancel_order(Sell[2])
            trail_sl_buy('US30',Buy[2])

        elif (mt5.positions_total() == 1) and (mt5.positions_get(symbol='US30')[0][0] == Sell[2]): # open sell position
            cancel_order(Buy[2])
            trail_sl_sell('US30',Sell[2])




In [ ]:
symbol = 'EURUSD.i'
order_types = (0,1)
first_order = random.choice(order_types)
pip_sl = 0.5
order_history = []


while True:
    first_order = random.choice(order_types)
    if first_order == 0:
        order = sell(symbol,0.9,pip_sl)
        sleep(1)
        first_sl = order.request.sl
        try:
            while mt5.positions_total() > 0:
                R = (entry_price - mt5.symbol_info_tick(symbol).ask) // (pip_sl*10*mt5.symbol_info(symbol).point)
                if R >= 2:
                    new_sl= first_sl - R*pip_sl*10*mt5.symbol_info(symbol).point
                    if new_sl < mt5.positions_get()[0][11]:
                        respond = trail_sl_sell(symbol,new_sl)
                        print(respond,'---','R:',R-1)
        except IndexError:
            continue

    elif first_order == 1:
        order = buy(symbol,0.9,pip_sl)
        sleep(1)
        entry_price = order.request.price
        first_sl = order.request.sl
        try:
            while mt5.positions_total() > 0:
                R = (mt5.symbol_info_tick(symbol).bid - entry_price) // (pip_sl*10*mt5.symbol_info(symbol).point)
                if R >= 2:

                    new_sl= first_sl + R*pip_sl*10*mt5.symbol_info(symbol).point
                    if new_sl > mt5.positions_get()[0][11]:
                        respond = trail_sl_buy(symbol,new_sl)
                        print(respond,'---','R:',R-1)
        except IndexError:
            continue